In [1]:
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import nltk
from rouge import Rouge
import time

In [2]:
review = pd.read_csv('../../datasets/cleaned_data.csv', encoding='utf-8')
print(review.shape)
df = review[['headlines','text']]
df.head()

(55104, 3)


,headlines,text
0,startup unveils plan seater electric passenger...,usbased startup wright electric unveiled plans...
1,pm inrates st phase sauni project gujarat,pm narendra modi tuesday inrated first phase c...
2,trailer themma watsons circle released,trailer themma watson tom hanks starrer americ...
3,wont share even drop water states,ahead assthembly polls punjab chief minister p...
4,centre released cr mgnrega year,centre released crore states mahatma gandhi na...


In [3]:
!pip install transformers
!pip install sentence-transformers
!pip install rouge
!pip install rouge-score

In [4]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [11]:
dicto={'Text':[],'Original_summary':[],'Predicted_summary':[],'bleu':[],'Semantic_similarity':[],'rouge_1_f':[],'rouge_2_f':[],'rouge_l_f':[]}
for i in tqdm(range(100)):
    dicto['Text'].append(df.text[i])
    dicto['Original_summary'].append(df.headlines[i])
    dicto['Predicted_summary'].append(summarizer(df.text[i], max_length=20, min_length=5, do_sample=False)[0]['summary_text'])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:05<00:00,  2.45s/it]


In [12]:
summs_df = pd.DataFrame(list(zip(dicto['Text'], dicto['Original_summary'], dicto['Predicted_summary'])), columns=['Text', 'Original_summary', 'Predicted_summary'])
len(summs_df)

100

In [13]:
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('stsb-roberta-large')

In [14]:
rouge = Rouge()

for i in tqdm(range(0,len(summs_df))):

  original_summ = summs_df.Original_summary[i]
  predicted_summ = summs_df.Predicted_summary[i]

  embedding1 = sentence_model.encode(original_summ, convert_to_tensor=True)
  embedding2 = sentence_model.encode(predicted_summ, convert_to_tensor=True)
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

  rouge_score = rouge.get_scores(original_summ, predicted_summ, avg=True)

#   scores_dict['Semantic_similarity'].append(cosine_scores.item())
#   scores_dict['bleu'].append(nltk.translate.bleu_score.sentence_bleu(original_summ, predicted_summ))
#   scores_dict['rouge_1_f'].append(rouge_score['rouge-1']['f'])
#   scores_dict['rouge_2_f'].append(rouge_score['rouge-2']['f'])
#   scores_dict['rouge_l_f'].append(rouge_score['rouge-l']['f'])

  dicto['Semantic_similarity'].append(cosine_scores.item())
  dicto['bleu'].append(nltk.translate.bleu_score.sentence_bleu(original_summ, predicted_summ))
  dicto['rouge_1_f'].append(rouge_score['rouge-1']['f'])
  dicto['rouge_2_f'].append(rouge_score['rouge-2']['f'])
  dicto['rouge_l_f'].append(rouge_score['rouge-l']['f'])

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\swaga\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\swaga\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\swaga\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how

In [15]:
summs_df = pd.DataFrame(list(zip(dicto['Text'], dicto['Original_summary'], dicto['Predicted_summary'], dicto['Semantic_similarity'], dicto['bleu'], dicto['rouge_1_f'], dicto['rouge_2_f'], dicto['rouge_l_f'])), columns=['Text', 'Original_summary', 'Predicted_summary', 'Semantic_similarity', 'bleu', 'rouge_1_f', 'rouge_2_f', 'rouge_l_f'])
summs_df.head()

,Text,Original_summary,Predicted_summary,Semantic_similarity,bleu,rouge_1_f,rouge_2_f,rouge_l_f
0,usbased startup wright electric unveiled plans...,startup unveils plan seater electric passenger...,usbased startup wright electric unveiled plan...,0.763150,1.088159e-231,0.272727,0.100000,0.285714
1,pm narendra modi tuesday inrated first phase c...,pm inrates st phase sauni project gujarat,Inrated first phase crore saurashtra narmada ...,0.570590,1.239529e-231,0.250000,0.000000,0.250000
2,trailer themma watson tom hanks starrer americ...,trailer themma watsons circle released,american science fiction drama film circle re...,0.273789,1.132699e-231,0.235294,0.133333,0.250000
3,ahead assthembly polls punjab chief minister p...,wont share even drop water states,Punjab chief minister parkash singh singh bad...,0.193488,1.156702e-231,0.222222,0.125000,0.235294
4,centre released crore states mahatma gandhi na...,centre released cr mgnrega year,Centre released crore states in crore states ...,0.343669,1.074966e-231,0.100000,0.000000,0.117647


In [16]:
summs_df.to_csv('../../datasets/Transformer_generated_summaries.csv',index=False)